In [9]:
import numpy as np
from ephys import core, events
import neuraltda.topology2 as tp2
from importlib import reload

In [2]:
# Set up birds and block_paths
birds = ['B1083', 'B1056', 'B1235', 'B1075']
bps = {'B1083': '/home/brad/krista/B1083/P03S03/', 'B1075': '/home/brad/krista/B1075/P01S03/',
       'B1235': '/home/brad/krista/B1235/P02S01/', 'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/',
       'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/'}


In [3]:
bp = bps['B1056']

In [4]:
spikes = core.load_spikes(bp)
trials = events.load_trials(bp)
fs = core.load_fs(bp)
clusters = core.load_clusters(bp)

In [5]:
trial = 0
stim_trials = trials[trials['stimulus'] == 'D_scaled_burung']
trial_bounds = (stim_trials.iloc[trial]['time_samples'], stim_trials.iloc[trial]['stimulus_end'])
clusters_to_use = clusters[clusters['quality'] == 'Good']
clusters_list = clusters_to_use['cluster'].unique()
spikes = spikes[spikes['cluster'].isin(list(clusters_list))]

In [27]:
windows = np.around(np.linspace(0, 5, 123)*fs)
segment = tp2.get_segment((0, trial_bounds[1]-trial_bounds[0]), fs, [0,0])
nclus = len(clusters_list)


In [17]:
reload(tp2)
test = tp2.build_poptens_given_windows(stim_trials, spikes, windows, 0, clusters_list, nclus, segment)

In [18]:
np.amax(test)

0.0

In [36]:
nreps = len(stim_trials.index)
stim_recs = stim_trials['recording'].values 
nwins = len(windows)
poptens = np.zeros((nclus, nwins, nreps))
for rep in range(nreps):
    trial_start = stim_trials.iloc[rep]['time_samples']
    trial_end = stim_trials.iloc[rep]['stimulus_end']
    samp_period = (trial_start + segment[0], 
                   trial_start + segment[1])
    rec = stim_recs[rep]
    stim_rec_spikes = tp2.get_spikes_in_window(spikes, samp_period, rec)
    clusters = stim_rec_spikes['cluster'].values 
    sptimes = stim_rec_spikes['time_samples'].values - samp_period[0]
    clusters = stim_rec_spikes['cluster'].values
    sptclur = np.tile(sptimes[:, np.newaxis], (1, nwins))
    swin = np.tile(windows[np.newaxis, :], (len(sptimes), 1))
    upper = (sptclur <= swin)[:, 1:]
    lower = (sptclur > swin)[:, 0:-1]
    binss = np.logical_and(upper, lower)
    for clu in clusters_list:
        poptens[clusters_list==clu, 1:, rep] = np.sum(binss[clusters==clu, :], axis=0)

In [37]:
poptens[:, :, 0]

array([[0., 0., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 2., ..., 0., 0., 0.],
       [0., 3., 2., ..., 0., 0., 0.]])

In [30]:
np.linspace(0,5,123)

array([0.        , 0.04098361, 0.08196721, 0.12295082, 0.16393443,
       0.20491803, 0.24590164, 0.28688525, 0.32786885, 0.36885246,
       0.40983607, 0.45081967, 0.49180328, 0.53278689, 0.57377049,
       0.6147541 , 0.6557377 , 0.69672131, 0.73770492, 0.77868852,
       0.81967213, 0.86065574, 0.90163934, 0.94262295, 0.98360656,
       1.02459016, 1.06557377, 1.10655738, 1.14754098, 1.18852459,
       1.2295082 , 1.2704918 , 1.31147541, 1.35245902, 1.39344262,
       1.43442623, 1.47540984, 1.51639344, 1.55737705, 1.59836066,
       1.63934426, 1.68032787, 1.72131148, 1.76229508, 1.80327869,
       1.8442623 , 1.8852459 , 1.92622951, 1.96721311, 2.00819672,
       2.04918033, 2.09016393, 2.13114754, 2.17213115, 2.21311475,
       2.25409836, 2.29508197, 2.33606557, 2.37704918, 2.41803279,
       2.45901639, 2.5       , 2.54098361, 2.58196721, 2.62295082,
       2.66393443, 2.70491803, 2.74590164, 2.78688525, 2.82786885,
       2.86885246, 2.90983607, 2.95081967, 2.99180328, 3.03278